# 3. Shell és Regex

>Számítógépes nyelvészet, 2018 tavasz

>Simon Eszter, Mittelholcz Iván

>MTA, Nyelvtudományi Intézet

## Tartalom

* [1. Shell](#1.-Shell)
    * [1.1. Alapfogalmak](#1.1.-Alapfogalmak)
    * [1.2. Történelem](#1.2.-Történelem)
    * [1.3. Hogyan működik](#1.3.-Hogy-működik?)
    * [1.4. Fájlkezelés](#1.4.-Fájlkezelés)
    * [1.5. Tulajdonságok](#1.5.-Tulajdonságok)
    * [1.6. Fontosabb parancsok](#1.6.-Fontosabb-parancsok)
* [2. Regex](#2.-Regex)
* [3. Irodalom](#3.-Irodalom)

## 1. Shell

### 1.1. Alapfogalmak

Kernel (rendszermag): erőforrások kezelése, absztrakció

* feladatütemezés (folyamatok hozzáférése a CPU-hoz)
* memória (RAM) kezelés
* fájlrendszer
* I/O
* stb.

Alkalmazások:

* külön programok
* a kernelen keresztül kommunikálnak másokkal

Shell (rendszerhéj): karakteres felhasználói felület alkalmazások futtatására

* maga is alkalmazás, több féle van, cserélhető
    * mostanában Linux-ok többségén és OS X-en a [__bash__](https://www.gnu.org/software/bash/) az alapértelmezett
* programozási nyelv (interpretált)

In [10]:
%%bash
 ls -la

total 44
drwxr-xr-x 5 mittelholcz mittelholcz  4096 febr  14 13:18 .
drwxr-xr-x 6 mittelholcz mittelholcz  4096 febr   6 16:06 ..
-rw-r--r-- 1 mittelholcz mittelholcz    33 febr  14 11:28 hello.sh
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr   6 16:10 .ipynb_checkpoints
-rw-r--r-- 1 mittelholcz mittelholcz 18550 febr  14 13:18 shell_regex.ipynb
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train1
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train2


### 1.2. Történelem

* 1969: UNIX oprendszer. Kezdetben hozzávágják mindenkihez, aki gépet vesz, aztán rájönnek, hogy a szoftver is lehet üzlet $\to$ zárt lesz.
* 1983: GNU (GNU's Not Unix). UNIX-szerű, de nem tartalmaz UNIX-ból származó kódot. Open source. Kernel nincs (Hurd), minden más van (C fordító, debugger és library, shell, make, TeX, ablak kezelő, stb.).
* 1991: Linux kernel.
* Azóta:
    * __GNU/Linux__. Linux kernel + GNU eszközök + mindenféle más. Röviden ezt nevezik Linux-nak.
    * __UNIX-like__ vagy __\*NIX__: olyan rendszerek, amik _hasonlóan néznek ki_, mint a UNIX (GNU/Linux, BSD, OS X, stb.)

### 1.3. Hogy működik?

**Keresési sorrend**

* _alias_-ok között (`alias <név>='<utasítások>'`, pl. `alias ll='ls -l'`)
* _builtin_-ek között (pl. `alias`, `type`, `echo`, bővebben l. [itt](https://www.gnu.org/software/bash/manual/html_node/Bash-Builtins.html))
* *alkalmazás*ok között (pl. `ls`, `grep`, stb.)

Mi számít külső alkalmazásnak? Ami a `$PATH` változóban felsorolt könyvtárakban van.

In [11]:
%%bash

echo $PATH

/home/mittelholcz/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/lib/jvm/java-8-oracle/bin:/usr/lib/jvm/java-8-oracle/db/bin:/usr/lib/jvm/java-8-oracle/jre/bin


**Változók**

* minden string
* a szóköz delimiter, utána új kifejezés kezdődik, ezért
    * értékadásban az `=` körül nincs szóköz
    * ha az érték szóközt tartalmaz, akkor idézőjel kell
* duplaidézőjelnél van helyettesítés, szimplánál nincs
* hivatkozás `$<változó>` vagy `${<változó>}` formában

In [59]:
%%bash

# értékadás
a=alma #barack
b="$a barack"
c='$a citrom'

# hivatkozás
echo $a
echo $b
echo $c

alma
alma barack
$a citrom


**Alkalmazás indítása**

Ha külső parancsot kell végrehajtania, a shell

1. létrehozza saját maga klónját (fork) -- gyerekfolyamat
2. a forkban futtatja a kapott parancsot (exec).

$\to$ Semmilyen változás nem terjed visszafelé, minden csak a gyerekfolyamatokban érvényes (pl. PATH megváltoztatása, új alias, stb). Ez a shell szkriptekre is igaz: azok is új shell-ben futnak, nincs hatásuk az indíto shell-re.

In [60]:
%%bash
echo 'eredeti:' $valtozo
bash
valtozo='érték'
echo 'uj:' $valtozo
exit
echo 'eredeti:' $valtozo

eredeti:
uj: érték
eredeti:


**Gyakorlat**

1. Írjunk Python shell-t!
2. Írjunk shell-t!

### 1.4. Fájlkezelés

Alapvető parancsok

* `ls`: könyvtár tartalmának listázása
* `mkdir <könyvtár>`: könyvtár létrehozása
* `rmdir <könyvtár>`: könyvtár törlése
* `cp <honnan> <hova>`: fájl/könyvtár másolása
* `mv <mit> <mire>`: fájl/könyvtár átnevezése/mozgatása 
* `rm <fájl>`: törlés
* `pwd`: aktuális könyvtár

Kiemelt könyvárak rövidítései:

* `.`: jelenlegi könyvtár
* `..`: szülő könyvtár
* `/`: gyökér könyvtár

A `..` segítségével lehet relatív elérési utat megadni visszafelé is.

### 1.5. Tulajdonságok

**Programok indítása, paraméterezése**

paraméterek:

* opcionálisak vagy kötelezőek
* rövidítettek vagy hosszúak (pl. `-a` vagy `--all`)
* értéktelenek vagy értékesek esetleg opcionálisan értékesek (pl. `--color` vagy `--color=never`).

In [14]:
%%bash
ls
echo
ls --all -l
echo
ls -la
echo
ls -la --color=always

hello.sh
shell_regex.ipynb
train1
train2

total 44
drwxr-xr-x 5 mittelholcz mittelholcz  4096 febr  14 13:26 .
drwxr-xr-x 6 mittelholcz mittelholcz  4096 febr   6 16:06 ..
-rw-r--r-- 1 mittelholcz mittelholcz    33 febr  14 11:28 hello.sh
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr   6 16:10 .ipynb_checkpoints
-rw-r--r-- 1 mittelholcz mittelholcz 19206 febr  14 13:26 shell_regex.ipynb
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train1
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train2

total 44
drwxr-xr-x 5 mittelholcz mittelholcz  4096 febr  14 13:26 .
drwxr-xr-x 6 mittelholcz mittelholcz  4096 febr   6 16:06 ..
-rw-r--r-- 1 mittelholcz mittelholcz    33 febr  14 11:28 hello.sh
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr   6 16:10 .ipynb_checkpoints
-rw-r--r-- 1 mittelholcz mittelholcz 19206 febr  14 13:26 shell_regex.ipynb
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train1
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 trai

**Saját programok futtatása**

* [shebang](https://en.wikipedia.org/wiki/Shebang_%28Unix%29)

In [15]:
%%bash
cat hello.sh

#! /bin/bash
echo 'Helló héj!'


In [16]:
%%bash
bash hello.sh

Helló héj!


In [17]:
%%bash
./hello.sh

bash: line 1: ./hello.sh: Permission denied


**Jogosultságkezelés**

Jogok:

* olvasási (**r**)
* írási (**w**)
* végrehajtási (**x**)

Felhasználók:

* Tulajdonos (**u**)
* Csoport (**g**)
* Mások (**o**)

Parancsok:

* `chmod`: jogosultság megváltoztatása
* `chown`: tulajdonos / csoport megváltoztatása

In [50]:
%%bash
chmod u+x hello.sh
./hello.sh

Helló héj!


In [51]:
%%bash

# futtatás máshonnan
cd ..
./03.ShellRegex/hello.sh
cd  03.ShellRegex

Helló héj!


**Parancsbehelyettesítés**

`` `<parancs>` `` vagy `$(<parancs>)` formában

In [52]:
%%bash

# futtatás abszolút elérési úttal
/`pwd`/hello.sh

Helló héj!


**Wildcard-ok**

* `?` egy tetszőleges karakter helyett álhat
* `*` bárhány bármi helyett állhat
* `[chars]`: a szögletes zárójelben megadott karakterek valamelyike helyett állhat, tartomány is használható (pl. `[A-Z]` egy nagybetűt helyettesíthet)

In [53]:
%%bash
ls ?hell*

shell_regex.ipynb


**Pipe**

A`|` karakterrel lehet egy parancs kimenetét átadni a következő parancsnak bemenetként.

In [54]:
%%bash

ls | wc

      4       4      41


**Átirányítás**

Standard kommunikációs csatornak:

* stdin
* stdout
* stderr

Átirányítások:

* `command <filename`: fájl $\to$ stdin
* `command >filename`: stdout $\to$ fájl (felülír)
* `command >>filename`: stdout $\to$ fájl (appendál)
* `command 2>filename`: stderr $\to$ fájl

Speciális fájl a `/dev/null`, ami minden adatot "elnyel". Pl. a `command 2>/dev/null` minden hibaüzenetet eltüntet, csak a "rendes" kimenet lesz olvasható.

In [55]:
%%bash

# stdout és stdin
ls -la >list.txt
cat <list.txt

# stderr
./hello.sh 2>err.txt
echo
cat err.txt

total 48
drwxr-xr-x 5 mittelholcz mittelholcz  4096 febr  14 13:39 .
drwxr-xr-x 6 mittelholcz mittelholcz  4096 febr   6 16:06 ..
-rwxr--r-- 1 mittelholcz mittelholcz    33 febr  14 11:28 hello.sh
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr   6 16:10 .ipynb_checkpoints
-rw-r--r-- 1 mittelholcz mittelholcz     0 febr  14 13:39 list.txt
-rw-r--r-- 1 mittelholcz mittelholcz 23810 febr  14 13:38 shell_regex.ipynb
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train1
drwxr-xr-x 2 mittelholcz mittelholcz  4096 febr  14 11:28 train2
Helló héj!



**Visszatérési érték, elágazás**

* 0: jó
* nem 0: valamilyen hibakód

In [56]:
%%bash

cat hello.sh
echo $?
cat nincsilyenfile
echo $?

#! /bin/bash
echo 'Helló héj!'
0
1


cat: nincsilyenfile: No such file or directory


* `cmd1 && cmd2`: `cmd2` csak akkor fut le, ha `cmd1` rendben lefutott
* `cmd1 || cmd2`: `cmd2` csak akkor fut le, ha `cmd1` elszállt
* `cmd1 ; cmd2`: `cmd2` mindenképp lefut `cmd1` után

In [58]:
%%bash

# elégazás (if-else) röviden

./hello.sh && echo "jól lefutott" || echo "jól nem futott le"
chmod u-x hello.sh
./hello.sh && echo "jól lefutott" || echo "jól nem futott le"

jól nem futott le
jól nem futott le


bash: line 4: ./hello.sh: Permission denied
bash: line 6: ./hello.sh: Permission denied


### 1.6. Fontosabb parancsok

**Infó**

* `<command> -h` vagy `--help`: rövid használati utasítás
* `man <command>`: ezzel lehet egy program manual-ját megnyitni


**Szövegmegjelenítés, statisztika**

* `echo <string>`: String stdout-ra írása.
    * `-n`: nem ír újsor karaktert a végére
    * `-e`: értelmezi a backslash-elt karaktereket (`\n`, `\t`, stb.)
* `cat`: Fájlok konkatenálása és stdout-ra írása.
* `head -n <num>`: Fájl vagy stdin első _n_ sorának megjelenítése (default 10).
* `tail -n <num>`: Fájl vagy stdin utolsó _n_ sorának megjelenítése (default 10).
* `less`: Fájl vagy stdin megnyitása olvasásra (a vim-hez hasonlóan, kersés előre a `/`, vissza a `?` karakterekkel lehetséges).
* `wc`: Sorok, szavak és karakterek számolása.
    * `-l`: sorok
    * `-w`: szavak
    * `-c`: karakterek 

**Stringmanipuláció**

* `grep <kifejezés>`: Fájl vagy a stdin szűrése, a kifejezést tartalmazó sorokra.
    * `-i`: ignore case
    * `-v`: a nem illeszkedő sorokat engedi át
    * `-r <könyvtár>`: rekurívan keres a könyvtár minden alkönyvtárjában
* `tr "<char1s>" "<char2s>"`: A stdin-en lecseréli `char1`-eket `char2`-kre. Több karakter is megadható egyszerre, pl. a `tr "ab" "xy"` az *a*-kat *x*-ekre, a *b*-ket *y*-okra cseréli le.
* `sed "s/mit/mire/g"`: Kifejezés keresése és cseréje fájlban vagy stdin-en.
* `sort`: Rendezés.
    * `-n`: numerikus a lexikális helyett (10 > 9)
    * `-r`: fordított sorrend
* `uniq`: Sorok egyelése. Csak az egymást követő azonos sorokat egyeli, ezért előtte szükséges lehet rendezni.
    * `-c`: a sorok elé írja, hány darab volt belőlük
* `cut`: Oszlopok szelektálása. Default: `TAB`-ok mentén.
    * `-d"<char>"`: határoló karakter (pl. `-d";"`, `-d" "`)
    * `-f<nums>`: oszlopszám (pl. `-f2`, `-f2,4`, `-f2-4`)

### 1.7. Gyakorlatok

ötletek

* abba $\to$ acdc
* római számok
* mohosagi csapdakat is ki kell talalni!
* esetleg precedencia szabalyokat is lehet gyakoroltatni (csillag > concat > unio)
* a nagy egymásra épülő gyakorlat sorozat: tokenizálás $\to$ szógyakoriság $\to$ stopwords-szűrés
* redundancia, normalforma-hiany
* megjelolt blokk + ref
    * grep: szóismétlés szurese
    * sed: szoismetelteto

**1. stopwords**

A `train/stopwordsX.txt` listákból készíts egy közös, minden szót egyszer tartalmazó, rendezett listát `stopwords-full.txt` néven.

**2. tokenizálás**

A `train/kant.txt` fájl tokenizálása: egy sor = egy szó, punktuációkat kidobni, üres sorokat törölni. Elég a stdout-ra írni.

**3. szógyakoriság**

A fenti tokenizálást átformálni szógyakorisági listává: egy sor egy szót és az ő gyakoriságát tartalmazza. Egy szó (type) csak egyszer forduljon elő. A lista gyakoriság szerint fordítottan legyen rendezve.

**4. stopwords lista alkalmazása**

A szógyakorisági listát szűrjük a `stopwords-full.txt`-vel

## 2. Regex

Valami.

## 3. Irodalom

Shell, Bash:

* Software Carpentry: [The Unix Shell](http://swcarpentry.github.io/shell-novice/)
* [Bash dokumentáció](https://www.gnu.org/software/bash/manual/)
* [Advanced Bash-Scripting Guide](http://tldp.org/LDP/abs/html/index.html)

Regex:

* Jeffrey Friedl: *Mastering Regular Expressions.* O'Reilly, 2009. ([link](http://shop.oreilly.com/product/9780596528126.do))
* [Python: Regular Expression HOWTO](https://docs.python.org/3.6/howto/regex.html)
* [Python: a re modul dokumentációja](https://docs.python.org/3.6/library/re.html)

***

In [61]:
%%bash

# takaritas
rm -f *.txt